In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2017','era_2017.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2017-01-01 ... 2017-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2017-01-01 ... 2017-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2017-01-01 00:00:00 1000.0         49.25    -124.85    94.351730  0.003852
                                            -124.60    96.584152  0.003900
                                            -124.35    97.273605  0.003948
                                            -124.10    97.568527  0.004028
                                            -123.85    98.656418  0.004121
...                                                          ...       ...
2017-12-31 21:00:00 1000.0         24.50    -68.10     77.557327  0.013878
                                            -67.85     77.498734  0.013919
                                            -67.60     77.658890  0.013955
                                            -67.35     77.854202  0.014019
                                            -67.10     78.399124  0.014134

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2017-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2017-12-31 1000.0         49.25    -124.85    81.688110  0.006041
                                   -124.60    78.840744  0.006061
                                   -124.35    75.947525  0.005956
                                   -124.10    73.252441  0.005852
                                   -123.85    70.337563  0.005844
...                                                 ...       ...
                          24.50    -68.10     77.671211  0.014580
                                   -67.85     77.623573  0.014563
                                   -67.60     77.574158  0.014543
                                   -67.35     77.505913  0.014518
                                   -67.10     77.472656  0.014498

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2017-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 81.69 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006041 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:08 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2017_county_shapefile','cb_2017_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,005,00161528,0500000US01005,01005,Barbour,06,2292144656,50538698,"POLYGON ((-85.74803 31.61918, -85.74544 31.618..."
1,01,023,00161537,0500000US01023,01023,Choctaw,06,2365869837,19144469,"POLYGON ((-88.47323 31.89386, -88.46888 31.930..."
2,01,035,00161543,0500000US01035,01035,Conecuh,06,2201948618,6643480,"POLYGON ((-87.42720 31.26436, -87.42551 31.268..."
3,01,051,00161551,0500000US01051,01051,Elmore,06,1601762124,99965171,"POLYGON ((-86.41333 32.75059, -86.37115 32.750..."
4,01,065,00161558,0500000US01065,01065,Hale,06,1667907107,32423356,"POLYGON ((-87.87046 32.76244, -87.86818 32.765..."
...,...,...,...,...,...,...,...,...,...,...
3228,37,069,01008553,0500000US37069,37069,Franklin,06,1273631713,7304032,"POLYGON ((-78.54551 36.05670, -78.54493 36.077..."
3229,48,317,01383941,0500000US48317,48317,Martin,06,2369724595,1931832,"POLYGON ((-102.21103 32.17704, -102.21111 32.3..."
3230,54,107,01560558,0500000US54107,54107,Wood,06,948592039,27228519,"POLYGON ((-81.75582 39.18052, -81.75575 39.180..."
3231,13,269,00344156,0500000US13269,13269,Taylor,06,975612265,7802363,"MULTIPOLYGON (((-84.05331 32.52202, -84.00849 ..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-85.74803 31.61918, -85.74544 31.618...
1       POLYGON ((-88.47323 31.89386, -88.46888 31.930...
2       POLYGON ((-87.42720 31.26436, -87.42551 31.268...
3       POLYGON ((-86.41333 32.75059, -86.37115 32.750...
4       POLYGON ((-87.87046 32.76244, -87.86818 32.765...
                              ...                        
3228    POLYGON ((-78.54551 36.05670, -78.54493 36.077...
3229    POLYGON ((-102.21103 32.17704, -102.21111 32.3...
3230    POLYGON ((-81.75582 39.18052, -81.75575 39.180...
3231    MULTIPOLYGON (((-84.05331 32.52202, -84.00849 ...
3232    POLYGON ((-88.02858 30.22676, -88.02399 30.230...
Name: geometry, Length: 3233, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -85.748032  31.619181
     1   -85.745435  31.618898
     2   -85.742651  31.621259
     3   -85.741740  31.619403
     4   -85.739813  31.621810
...             ...        ...
3232 799 -87.999996  30.225753
     800 -88.013607  30.223490
     801 -88.021364  30.223334
     802 -88.027678  30.223645
     803 -88.028580  30.226763

[1040321 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3233): ## A total of 3232 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 5min 53s
Wall time: 6min 1s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,70.804562,0.010842
1,67.012844,0.010424
2,71.38154,0.011091
3,70.877697,0.010546
4,66.247463,0.010158
...,...,...
3228,67.282215,0.008994
3229,50.271697,0.007711
3230,70.438558,0.007879
3231,68.187833,0.01019


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065
...,...,...,...
3228,Franklin,"POLYGON ((-78.54551 36.05670, -78.54493 36.077...",37069
3229,Martin,"POLYGON ((-102.21103 32.17704, -102.21111 32.3...",48317
3230,Wood,"POLYGON ((-81.75582 39.18052, -81.75575 39.180...",54107
3231,Taylor,"MULTIPOLYGON (((-84.05331 32.52202, -84.00849 ...",13269


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005,70.804562,0.010842
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,67.012844,0.010424
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035,71.38154,0.011091
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051,70.877697,0.010546
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065,66.247463,0.010158
...,...,...,...,...,...
3228,Franklin,"POLYGON ((-78.54551 36.05670, -78.54493 36.077...",37069,67.282215,0.008994
3229,Martin,"POLYGON ((-102.21103 32.17704, -102.21111 32.3...",48317,50.271697,0.007711
3230,Wood,"POLYGON ((-81.75582 39.18052, -81.75575 39.180...",54107,70.438558,0.007879
3231,Taylor,"MULTIPOLYGON (((-84.05331 32.52202, -84.00849 ...",13269,68.187833,0.01019


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",1005,70.804562,0.010842
1,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,67.012844,0.010424
2,Conecuh,"POLYGON ((-87.42720 31.26436, -87.42551 31.268...",1035,71.38154,0.011091
3,Elmore,"POLYGON ((-86.41333 32.75059, -86.37115 32.750...",1051,70.877697,0.010546
4,Hale,"POLYGON ((-87.87046 32.76244, -87.86818 32.765...",1065,66.247463,0.010158
...,...,...,...,...,...
3228,Franklin,"POLYGON ((-78.54551 36.05670, -78.54493 36.077...",37069,67.282215,0.008994
3229,Martin,"POLYGON ((-102.21103 32.17704, -102.21111 32.3...",48317,50.271697,0.007711
3230,Wood,"POLYGON ((-81.75582 39.18052, -81.75575 39.180...",54107,70.438558,0.007879
3231,Taylor,"MULTIPOLYGON (((-84.05331 32.52202, -84.00849 ...",13269,68.187833,0.01019


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Barbour,1005,70.804562298216709,0.010841679595065
1,Choctaw,1023,67.012844366573390,0.010424327930299
2,Conecuh,1035,71.381540469757326,0.011091355373639
3,Elmore,1051,70.877696978117768,0.010546455051048
4,Hale,1065,66.247462904675359,0.010158325028945
...,...,...,...,...
3228,Franklin,37069,67.282215292729276,0.008993618644580
3229,Martin,48317,50.271697154985745,0.007710535962074
3230,Wood,54107,70.438557967258177,0.007878859196285
3231,Taylor,13269,68.187832890380278,0.010189755648399


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
3100,Autauga,1001,68.593856807648550,0.010388895180825
3232,Baldwin,1003,74.515771968995978,0.012141361522969
0,Barbour,1005,70.804562298216709,0.010841679595065
2212,Bibb,1007,69.578241146293379,0.010284876628906
758,Blount,1009,71.381569351937046,0.010106507627071
...,...,...,...,...
2211,Sweetwater,56037,56.169955060768125,0.003973206437771
933,Teton,56039,68.745028617984659,0.003928170644462
1566,Uinta,56041,57.333078736089952,0.003946129540677
2356,Washakie,56043,56.571321286438561,0.004133560825134


In [21]:
county_var.to_pickle('era_2017.pkl')